In [5]:
import h5py

# Path to your .h5 file
file_path = "20211005_1_0.h5"

# Open the .h5 file in read mode
with h5py.File(file_path, "r") as h5_file:
    # Function to recursively print the structure of the file
    def print_structure(name, obj):
        if isinstance(obj, h5py.Dataset):
            print(f"Dataset: {name}, Shape: {obj.shape}, Data Type: {obj.dtype}")
        elif isinstance(obj, h5py.Group):
            print(f"Group: {name}")
    
    print("File structure:")
    h5_file.visititems(print_structure)
    
    # Access a specific dataset (replace 'your_dataset' with the actual name)
    if 'your_dataset' in h5_file:
        data = h5_file['your_dataset'][:]
        print("\nDataset contents:")
        print(data)


File structure:
Dataset: data, Shape: (210, 2, 480, 640), Data Type: float32


In [ ]:
import h5py
import numpy as np
import time

# Define the range of file indices
file_range = range(60)  # From 0 to 59, for files like 20211005_1_0 to 20211005_1_59

# Loop through each file in the range
for i in file_range:
    # Construct the file paths
    input_file = f"20211005_1_{i}.h5"  # Adjust the input file name as needed
    output_file = f"20211005_1_{i}.txt"  # Adjust the output file name

    # Open the .h5 file and save the dataset to a .txt file
    with h5py.File(input_file, "r") as h5_file:
        # Access the dataset (replace 'data' with your actual dataset name)
        dataset = h5_file["data"][:]

        # Print the shape of the dataset to inspect it
        print(f"Shape of dataset in {input_file}: {dataset.shape}")

        # Determine the shape and process it accordingly
        if len(dataset.shape) == 3:
            # If dataset has 3 dimensions, reshape to (timestamps, height, width)
            data = dataset.reshape(dataset.shape[0], 2, dataset.shape[1], dataset.shape[2])
        elif len(dataset.shape) == 4:
            # If dataset already has 4 dimensions, no reshape needed
            data = dataset
        else:
            # Handle any unexpected dimensions
            print(f"Unexpected dataset shape in {input_file}. Skipping reshaping.")
            continue

    print(f"Data from the dataset 'data' in {input_file} has been processed.")

    # Define a function to extract events
    def extract_events(data):
        events = []
        timestamps = data.shape[0]  # Number of timestamps
        for t in range(timestamps):
            for x in range(data.shape[2]):
                for y in range(data.shape[3]):
                    # Check for changes in polarity (assuming channel 0 and channel 1 represent polarity)
                    polarity_change = data[t, 0, x, y] != data[t, 1, x, y]
                    if polarity_change:
                        polarity = 1 if data[t, 0, x, y] > data[t, 1, x, y] else -1
                        events.append((t, x, y, polarity))
        return events

    # Extract events from the data
    events = extract_events(data)

    # Get the current Unix timestamp as the base (you can replace this with a specific timestamp)
    base_unix_time = time.time()  # This will return the current time as a float

    # Save the events in the desired format
    with open(output_file, "w") as f:
        f.write("timestamp position(x y) polarity\n")
        for event in events:
            # Map the event index (t) to a Unix timestamp (now with float precision)
            timestamp = base_unix_time + event[0]  # You can adjust this logic if needed
            
            # Normalize polarity: Convert -1 to 0 and 1 stays as 1
            normalized_polarity = 0 if event[3] == -1 else 1
            
            # Write the position in "x y" format (space-separated)
            f.write(f"{timestamp:.6f} {event[1]} {event[2]} {normalized_polarity}\n")

    print(f"Formatted and normalized data for {input_file} has been saved to {output_file}")

print("All files have been processed and saved.")


Shape of dataset in 20211005_1_0.h5: (210, 2, 480, 640)
Data from the dataset 'data' in 20211005_1_0.h5 has been processed.
Formatted and normalized data for 20211005_1_0.h5 has been saved to 20211005_1_0.txt
Shape of dataset in 20211005_1_1.h5: (206, 2, 480, 640)
Data from the dataset 'data' in 20211005_1_1.h5 has been processed.
Formatted and normalized data for 20211005_1_1.h5 has been saved to 20211005_1_1.txt
Shape of dataset in 20211005_1_2.h5: (214, 2, 480, 640)
Data from the dataset 'data' in 20211005_1_2.h5 has been processed.
Formatted and normalized data for 20211005_1_2.h5 has been saved to 20211005_1_2.txt
Shape of dataset in 20211005_1_3.h5: (209, 2, 480, 640)
Data from the dataset 'data' in 20211005_1_3.h5 has been processed.
Formatted and normalized data for 20211005_1_3.h5 has been saved to 20211005_1_3.txt
Shape of dataset in 20211005_1_4.h5: (207, 2, 480, 640)
Data from the dataset 'data' in 20211005_1_4.h5 has been processed.
Formatted and normalized data for 202110